# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [799]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import math
from copy import copy
from dataclasses import dataclass
from random import random, choice, randint


## The *Nim* and *Nimply* classes

In [800]:
Nimply = namedtuple("Nimply", "row, num_objects")
genoma = {"rows": 0, "elements": 0}
evolve_info ={"genoma" : genoma, "sigma_r": 0.2, "sigma_t": 0.3}

NUM_GENERATION = 10
NUM_POPULATION = 50
NUM_AGENT_TO_MUTATE = 10
MUTATION = 0.15
NUM_PARENTS = 2
#ERE = 20
#LEARNING_RATE = 1/math.sqrt(ERE)


In [801]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [802]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [803]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [804]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = choice(spicy_moves)
    return ply


In [805]:
def agent_startegy(state: Nim, agent_genotype ) -> Nimply:
    
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    (row,tessere) = Nimply(*max(possible_moves, key=lambda m: ((m[0]+agent_genotype[0]), (m[1]+agent_genotype[1]))))
    if row > state.rows[0] and  tessere < 0:
        return Nimply(*max(possible_moves, key=lambda m: ((m[0]), (m[1]))))
    elif row > state.rows[0]:
        return Nimply(*max(possible_moves, key=lambda m: ((m[0]), (m[1]+agent_genotype[1]))))
    elif tessere < 0:
        return Nimply(*max(possible_moves, key=lambda m: ((m[0]+agent_genotype[0]), (m[1]+agent_genotype[1]))))
    return Nimply(*max(possible_moves, key=lambda m: ((m[0]+agent_genotype[0]), (m[1]+agent_genotype[1]))))

In [806]:
def play(agent_genotype, opponent):
    #strategy = (pure_random, agent_startegy)
    win  = 0
    for _ in range(0,100):
        player = 0
        nim = Nim(5)
        while nim:
            #logging.info(f"init : {nim}")
            if player == 0:
                ply=opponent(nim)
            else:
                ply=agent_startegy(nim, agent_genotype)
            #ply = strategy[player](nim)
            #logging.info(f"ply: player {player} plays {ply}")
            nim.nimming(ply)
            #logging.info(f"status: {nim}")
            player = 1 - player
        #logging.info(f"status: Player {player} won!")
        if player == 1:
            win +=1
        #print("WINNER")
    #print(win/100)
    return win/100


In [807]:
@dataclass
class Agent:
    fitness: float
    genotype: list[int]

In [808]:
def select_parent(pop):
    pool = [choice(pop) for _ in range(NUM_PARENTS)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

In [809]:
def mutate(agent: Agent, opponent) -> Agent:
    offspring = copy(agent)
    gene_to_mutate = randint(0, len(offspring.genotype)-1)#choice(offspring.genotype[0], offspring.genotype[1])
    mutation = choice((1,-1))
    offspring.genotype[gene_to_mutate] = offspring.genotype[gene_to_mutate] + mutation #not offspring.genotype[pos]
    offspring.fitness = play(offspring.genotype, opponent)
    return offspring

In [810]:
def xover(p1: Agent, p2: Agent):
        gene_from_p1 = randint(0, len(p1.genotype)-1)
        gene_from_p2 = 1-gene_from_p1
        return Agent(fitness=0, genotype=[p1.genotype[gene_from_p1], p2.genotype[gene_from_p2]])


In [811]:
def find_best_agent(opponent):
    population = [Agent(
        genotype =  [0,0], 
        fitness = 0,
        ) for _ in range(NUM_POPULATION)]
    for a in population:
        a.fitness = play(a.genotype, opponent)
    print("NEW ALGO\n")
    print(population)

    for generation in range(NUM_GENERATION):
        next_gen = list()
        for agent_to_mutate in range(NUM_AGENT_TO_MUTATE):
            if random() < MUTATION:  # self-adapt mutation probability
            # mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate(p, opponent)
            else:
                p1 = select_parent(population)
                p2 =select_parent(population)
                o = xover(p1,p2)
            next_gen.append(o)

        for a in next_gen:
             a.fitness = play(a.genotype, opponent)
        population.extend(next_gen)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:NUM_POPULATION]
        print(population[0].fitness)
    return population[0]


## Oversimplified match

In [812]:
nim = Nim(5)
#find_best_generation()
opponent = optimal
#best_agent = find_best_agent(opponent)
#print(f"THE BEST AGENT: {best_agent}")

test_agent  = Agent(fitness=0, genotype=[0,-2])
logging.getLogger().setLevel(logging.INFO)

#strategy = (optimal, pure_random)
nim = Nim(5)
#comma(nim)
logging.info(f"init : {nim}")
player = 0
win = 0
for i in range(100):
    nim = Nim(5)
    player = 0
    while nim:
        if player == 0:
            ply = opponent(nim)
        else:
            ply=agent_startegy(nim, test_agent.genotype)
        logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        logging.info(f"status: {nim}")
        player = 1 - player
    if player == 1:
        win+=1

logging.info(f"Best Agent won: {(win/100)*100}% of matches")
logging.info(f"Best Agent Genotype: {test_agent.genotype}")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=4)
INFO:root:status: <1 3 5 3 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 3 5 3 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 1 5 3 0>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 1 5 0 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <1 1 1 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 1 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <1 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=6)
INFO:root:status: <1 3 5 1 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 3 5 1 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <1 3